Title etc

In [1]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv
!pip install openai==0.28

#load the Open AI api key file
load_dotenv()
#read key
openai.api_key = os.getenv("OPENAI_API_KEY")

if openai.api_key is None:
    print("⚠️ WARNING: OpenAI API Key not loaded. Please ensure you have a '.env' file in your environment with 'OPENAI_API_KEY=...'")
else:
    print("✅ OpenAI API Key successfully loaded from .env file.")

✅ OpenAI API Key successfully loaded from .env file.


In [2]:
#Data Loading
FILE_PATH = '/final.csv'

try:
    # Load the uploaded file
    df = pd.read_csv(FILE_PATH, encoding='utf-8')

    # Data Check
    print("-" * 50)
    print("✅ DataFrame successfully loaded.")
    print("\nInitial Data Structure:")
    print(df.head())
    print("\nData Types:")
    df.info()

except FileNotFoundError:
    print(f"❌ Error: The file '{FILE_PATH}' was not found. Please ensure it is uploaded!!!")

--------------------------------------------------
✅ DataFrame successfully loaded.

Initial Data Structure:
              ProductID                                       Product Name  \
0  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
1  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
2  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
3  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
4  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   

                                            Category   Brand  Ratings  \
0  Electronics,iPad & Tablets,All Tablets,Fire Ta...  Amazon      5.0   
1  Electronics,iPad & Tablets,All Tablets,Fire Ta...  Amazon      5.0   
2  Electronics,iPad & Tablets,All Tablets,Fire Ta...  Amazon      5.0   
3  Electronics,iPad & Tablets,All Tablets,Fire Ta...  Amazon      4.0   
4  Electronics,iPad & Tablets,All Tablets,Fire Ta...  Ama

In [3]:
#Choose the category for analysis
# Assuming the main category is the first item in the comma-separated string
# For example: 'Electronics,iPad & Tablets,All Tablets...' -> 'Electronics'
categories = df['Category'].str.split(',').str[0].str.strip()

print("\n\n--- Common Top-Level Categories ---")
print(categories.value_counts().head())



--- Common Top-Level Categories ---
Category
Fire Tablets       24874
AA                 12071
Stereos             6621
Back To College     5056
Electronics         4160
Name: count, dtype: int64


Picked the first 3 categories: "Fire Tablets", "AA", "Stereos".

In [4]:
import pandas as pd
import numpy as np

#Define Target Categories
TARGET_CATEGORIES = ['Fire Tablets', 'AA', 'Stereos']
FILE_PATH = '/final.csv'

# Reload the DataFrame (ensure fresh start for this block)
try:
    df = pd.read_csv(FILE_PATH, encoding='utf-8')
except FileNotFoundError:
    print(f"❌ Error: The file '{FILE_PATH}' was not found. Please re-upload.")
    exit()

#Creating primary category column
df['Primary Category'] = df['Category'].str.split(',').str[0].str.strip()

#filter the data
df_filtered = df[df['Primary Category'].isin(TARGET_CATEGORIES)].copy()

#drop rows where reviews are missing.
df_filtered.dropna(subset=['reviews.text'], inplace=True)

#make sure that 'Ratings' is numeric (it might be loaded as object if there were non-numeric values)
df_filtered['Ratings'] = pd.to_numeric(df_filtered['Ratings'], errors='coerce')
df_filtered.dropna(subset=['Ratings'], inplace=True)

#aggregate data by product | Group by the primary category and the product name
df_aggregated = df_filtered.groupby(['Primary Category', 'Product Name']).agg(
    # Calculate the average rating for the product
    Average_Rating=('Ratings', 'mean'),
    # Concatenate all reviews into a single string (separated by a space)
    All_Reviews=('reviews.text', lambda x: ' '.join(x.astype(str))),
    # Count the total number of reviews for the product
    Review_Count=('reviews.text', 'size')
).reset_index()

#inspection of aggregated data
print("✅ Aggregation complete.")
print("-" * 60)
print("Aggregated DataFrame Head (Products and Combined Reviews):")
print(df_aggregated.head())
print("-" * 60)
print("Aggregated DataFrame Information:")
df_aggregated.info()

✅ Aggregation complete.
------------------------------------------------------------
Aggregated DataFrame Head (Products and Combined Reviews):
  Primary Category                                       Product Name  \
0               AA  AmazonBasics AA Performance Alkaline Batteries...   
1               AA  AmazonBasics AAA Performance Alkaline Batterie...   
2     Fire Tablets  All-New Fire 7 Tablet with Alexa, 7" Display, ...   
3     Fire Tablets  All-New Fire HD 8 Kids Edition Tablet, 8 HD Di...   
4     Fire Tablets  All-New Fire HD 8 Kids Edition Tablet, 8 HD Di...   

   Average_Rating                                        All_Reviews  \
0        4.453594  Bulk is always the less expensive way to go fo...   
1        4.448040  I order 3 of them and one of the item is bad q...   
2        4.585366  Amazing way to keep my kids reading and also t...   
3        4.630901  I have been wanting to get a tablet for my gra...   
4        4.641638  Purchase this Amazon - Fire HD 8 Kids 

The resulting df_aggregated DataFrame now contains

    Primary Category: To loop through and generate articles for each one.

    Product Name: The name of the product.

    Average_Rating: To rank the products (Top 3 and Worst).

    All_Reviews: A single text blob containing every review for that specific product, ready to be sent to GPT-3 for analysis and summarization.

    Review_Count: Used to ensure we only summarize products with a sufficient number of reviews.

In [5]:
#Prompt design

SYSTEM_PROMPT = """
You are a professional product reviewer and technical writer.
Your task is to analyze a large body of customer reviews and generate a well-structured, unbiased article summarizing(like a blog post) the best and worst products in a given category.
You MUST adhere strictly to the requested output structure.
"""

def generate_article_summary(category_df, model="gpt-3.5-turbo"):
    """
    Generates a structured article summary for a single product category.
    """
    if category_df.empty:
        return "No products found for this category."

#Prepare Input Data for the Prompt

    # 1. Rank Products
    # We will use the product with the lowest review count as a threshold to filter out outliers.
    min_reviews = category_df['Review_Count'].quantile(0.25) # Exclude bottom 25% of products by review count
    category_df_filtered = category_df[category_df['Review_Count'] >= min_reviews].copy()

    # Sort to find the Top 3 and Worst
    top_products = category_df_filtered.sort_values(by='Average_Rating', ascending=False).head(3)
    worst_product = category_df_filtered.sort_values(by='Average_Rating', ascending=True).head(1)

    # 2. Concatenate Reviews for Top Products and Worst Product
    top_reviews_text = " ".join(top_products['All_Reviews'].tolist())
    worst_reviews_text = worst_product['All_Reviews'].iloc[0] if not worst_product.empty else "No worst product identified."

    # 3. Format the product list for the prompt
    product_list = "\n".join(
        [f"- {row['Product Name']} (Avg Rating: {row['Average_Rating']:.2f})"
         for index, row in top_products.iterrows()]
    )
    category_name = category_df_filtered['Primary Category'].iloc[0]


    # --- B. Define the User Prompt ---
    USER_PROMPT = f"""
    Based on the provided customer reviews, generate a comprehensive article for the product category: '{category_name}'.

    **TASK REQUIREMENTS:**
    1.  **Top 3 Products:** Identify the top 3 best-rated products from the list below.
    2.  **Key Differences:** Summarize the main functional/feature differences between these top 3 products.
    3.  **Top Complaints:** For EACH of the top 3 products, list the 2-3 most frequent negative complaints (based on the combined review text).
    4.  **Worst Product:** Identify the single worst-rated product in the category.
    5.  **Avoidance Reason:** Summarize the key reasons (complaints/flaws) why the worst product should be avoided.

    **PRODUCTS & RATINGS (for ranking reference):**
    {product_list}

    **COMBINED REVIEW TEXT FOR TOP PRODUCTS (Use this for differences/complaints):**
    ---
    {top_reviews_text[:3000]}... [Truncated for API efficiency]
    ---

    **COMBINED REVIEW TEXT FOR WORST PRODUCT (Use this for avoidance reasons):**
    ---
    {worst_reviews_text[:1000]}... [Truncated for API efficiency]
    ---

    **FORMAT:**
    Present the output as a Markdown-formatted blog post, using H2 titles for sections.
    """

    # --- C. API Call ---
    try:
        print(f"⏳ Generating summary for {category_name}...")
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": USER_PROMPT}
            ],
            temperature=0.7 # A bit of creativity for a blog post tone
        )
        return response.choices[0].message['content']
    except Exception as e:
        return f"An API error occurred: {e}"

# --- D. Execution Loop (Next Block) ---
# Now, we are ready to loop through the categories and call the function.
categories = df_aggregated['Primary Category'].unique()
all_articles = {}

for category in categories:
    category_data = df_aggregated[df_aggregated['Primary Category'] == category]

    # Generate the article summary
    article_markdown = generate_article_summary(category_data)

    # Store and print the result
    all_articles[category] = article_markdown
    print(f"\n\n--- ARTICLE FOR: {category} ---\n")
    print(article_markdown)
    print("\n---------------------------------------------------------")


print("\n\n✅ ALL ARTICLES GENERATED!")

⏳ Generating summary for AA...


--- ARTICLE FOR: AA ---


## Top 3 Products:

1. **AmazonBasics AAA Performance Alkaline Batteries (36 Count)**
   - Average Rating: 4.45

## Key Differences:

The top 3 products in the AA category are all highly rated, but they do have some differences based on customer reviews. While the specific details may vary, one common aspect that stood out was the reliability and long-lasting performance of these batteries. The key differences lie in their packaging, price point, and overall customer satisfaction.

1. **AmazonBasics AAA Performance Alkaline Batteries (36 Count)**: This product stands out for its consistent performance and value for money. Customers appreciated the long shelf life and reliability of these batteries, making them suitable for various devices.

## Top Complaints:

### AmazonBasics AAA Performance Alkaline Batteries (36 Count):
1. Some customers reported receiving batteries with missing backup springs, leading to issues with battery

In [6]:
# Assuming the 'all_articles' dictionary from the previous step is still in memory.
# It contains: {Category Name: Markdown Article Text}

OUTPUT_FILENAME = 'project_articles.md'

try:
    with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
        print(f"Starting to write articles to {OUTPUT_FILENAME}...")

        for category, content in all_articles.items():
            # Write a large header for the category
            f.write(f"# Product Review: {category}\n\n")

            # Write the generated article content (which is already in Markdown)
            f.write(content)

            # Add a separator for the next article
            f.write("\n\n---\n\n")

    print(f"\n✅ Successfully saved all generated articles to '{OUTPUT_FILENAME}'.")
    print("You can download this file from your Colab file explorer.")

except Exception as e:
    print(f"❌ An error occurred while writing the file: {e}")

Starting to write articles to project_articles.md...

✅ Successfully saved all generated articles to 'project_articles.md'.
You can download this file from your Colab file explorer.
